In [1]:
import torch

import zipfile
from pathlib import Path

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
import datetime
import random
import cv2
import src.models as mdls
import src.dataloaders as dataloaders
import src.utils as utils

In [2]:
config = utils.get_options()

## for google colab

In [3]:
model = mdls.get_pretrained_VGG16()
dataloader = dataloaders.dataloader

/cephfs/projects/ppashin/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cephfs/projects/ppashin/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
print(len(dataloaders.dataloader))
print(len(dataloaders.dataset_for_detection))
print(len(dataloaders.dataset_of_backgrounds))

print(dataloaders.dataset_of_backgrounds[89])

46
3350
2500
(array([[[0.9647059 , 0.99215686, 1.        , ..., 0.7176471 ,
         0.7294118 , 0.7411765 ],
        [0.9764706 , 0.99607843, 0.99607843, ..., 0.6666667 ,
         0.6784314 , 0.6862745 ],
        [0.99215686, 1.        , 1.        , ..., 0.6509804 ,
         0.6509804 , 0.65882355],
        ...,
        [0.5176471 , 0.4862745 , 0.4745098 , ..., 0.45882353,
         0.45882353, 0.47058824],
        [0.5058824 , 0.4745098 , 0.46666667, ..., 0.45490196,
         0.45882353, 0.47058824],
        [0.5254902 , 0.49019608, 0.49019608, ..., 0.4745098 ,
         0.4745098 , 0.4862745 ]],

       [[0.95686275, 0.9882353 , 1.        , ..., 0.6666667 ,
         0.67058825, 0.68235296],
        [0.96862745, 0.99215686, 0.99607843, ..., 0.6117647 ,
         0.61960787, 0.627451  ],
        [0.9882353 , 0.99607843, 1.        , ..., 0.5921569 ,
         0.59607846, 0.6039216 ],
        ...,
        [0.45490196, 0.42745098, 0.41960785, ..., 0.36078432,
         0.35686275, 0.36862746]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
# Define a directory to save the checkpoints
checkpoint_dir = "checkpoints/"

# Ensure the directory exists
import os
os.makedirs(checkpoint_dir, exist_ok=True)

In [7]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.to(device=3)
loss_fn.to(device=3)
None

In [8]:
print(len(dataloader))
for batch in dataloader:
    # print(batch[0])
    print(batch[1])
    break

46
tensor([[  1.0000,   7.6800,  23.9739,  60.0320,  84.8469],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  1.0000, 133.9733,  51.6903, 185.1733, 111.3488],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  1.0000,  78.0771, 106.6667, 105.3605, 138.4490],
        [  1.0000,  92.8000,  40.6219, 117.9733,  69.2481],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  1.0000,  86.2921,  33.2800, 210.9363, 177.9200],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  1.0000, 194.3324,   1.3653, 247.8080,  63.8293],
        [  1.0000,  36.5714,  34.7429, 111.7091, 122.5143],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000],
        [  1.0000,  95.3926,   4.3477

In [9]:
epochs = config['epochs']

for epoch in range(epochs):
    epoch_loss = 0.0
    for sample in (pbar := tqdm(dataloader)):

        img, box = img, box = sample[0].to(device=3), sample[1].to(device=3)
        img = img.to(torch.float32)

        box = box.to(torch.float32)
        optimizer.zero_grad()
        pred = model(img)

        loss = loss_fn(pred, box)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}\tLoss: {epoch_loss / len(dataloader)}")
    checkpoint_filename = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

  7%|▋         | 3/46 [00:28<06:45,  9.44s/it]


KeyboardInterrupt: 

In [ ]:
max_split_size_mb